#### Script para recortar a base de Aptidão Solo por Município

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [1]:
# Módulos necessários
import os
import glob
import geopandas as gpd
import numpy as np

In [2]:
# Definir diretório principal
dirpath = input('Diretório principal: ')

In [3]:
# Definir o caminho da pasta com os arquivos corrigidos de Aptidão Solo
originais_path = input('Pasta com corrigidos: ')

In [4]:
# Definir pasta com os arquivos do CAR
car_path = input('Diretório do CAR: ')

In [5]:
# Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
limites = input('Caminho Municípios: ')

In [6]:
# Criar pasta para armazenar os arquivos de Aptidão Solo recortados por município
out_path = os.path.join(dirpath, 'Aptd_Edafo_Municipios')
os.makedirs(out_path, exist_ok=True)
# Lendo shapefile de Municípios
lim = gpd.read_file(limites)
# Reprojetando o shapefile para WGS84
lim = lim.to_crs(epsg=4326)
# Códigos dos municípios
names = [x for x in lim['CD_MUN']]
# Lista dos estados
estados = list(np.unique(lim['SIGLA_UF']))
# Cortar por município
for estado in estados:
    shapes = glob.glob(originais_path + f'**/*{estado}_corrigido.shp')
    # Selecionar GeoDataFrame por estado
    select = lim[lim['SIGLA_UF'] == f'{estado}']
    # Agrupar por município para ser possível iterar sob as feições
    agrupado = select.groupby('CD_MUN')
    for shape in shapes:
        for key,values in agrupado:
            aptd = gpd.read_file(shape)
            aptd = aptd.to_crs(epsg=4326)
            nome_arq = os.path.basename(shape).replace('_corrigido.shp', '')
            output = out_path + f"\{nome_arq}_{key}.shp"
            mun = lim[lim["CD_MUN"] == f"{key}"]
            geodf_clip = gpd.clip(aptd, mun, keep_geom_type=True)
            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))

<string>:24: SyntaxWarning: invalid escape sequence '\{'
<>:24: SyntaxWarning: invalid escape sequence '\{'
<string>:24: SyntaxWarning: invalid escape sequence '\{'
<>:24: SyntaxWarning: invalid escape sequence '\{'
C:\Users\YouX\AppData\Local\Temp\ipykernel_9184\3083365827.py:24: SyntaxWarning: invalid escape sequence '\{'
  output = out_path + f"\{nome_arq}_{key}.shp"


In [8]:
# Renomeando AREA_IMOVEL para seus respectivos municípios
municipios = []
# Percorre a pasta 'CAR' e suas subpastas
for root, dirs, files in os.walk(car_path):
    for dir in dirs:
        # Verifica se o nome da subpasta é um código de município
        if dir.startswith('SHAPE_'):
            municipio_codigo = dir.split('_')[1]
            municipios.append(municipio_codigo)
            # Constrói o caminho completo para a pasta 'AREA_IMOVEL'
            area_imovel_dir = os.path.join(root, dir, 'AREA_IMOVEL')
            # Verifica se a pasta 'AREA_IMOVEL' existe
            if os.path.exists(area_imovel_dir):
                # Renomeia os arquivos dentro de 'AREA_IMOVEL'
                for filename in os.listdir(area_imovel_dir):
                    if filename.startswith('AREA_IMOVEL'):
                        novo_nome = f'{municipio_codigo}_{filename}'
                        arquivo_antigo = os.path.join(area_imovel_dir, filename)
                        novo_arquivo = os.path.join(area_imovel_dir, novo_nome)
                        os.rename(arquivo_antigo, novo_arquivo)
                        print(f'Renomeado: {arquivo_antigo} -> {novo_arquivo}')
# Criar pasta para armazenar os arquivos finais de Declividade por município integrados com CAR
final_path = os.path.join(dirpath, 'Aptd_Edafo_Municipios_CAR')
os.makedirs(final_path, exist_ok=True)
# Integração com CAR
# Caminho dos vetores da Aptidão do Solo
edafo = glob.glob(out_path + fr'**/*.shp')
# Caminho dos vetores de limites dos imóveis
car = glob.glob(car_path + '**/*/*/*.shp')
caminho_edafo = []
caminho_car = []
for arquivo in edafo:
    caminho = os.path.join(out_path,arquivo)
    caminho_edafo.append(caminho)
for arquivo in car:
    caminho = os.path.join(car_path,arquivo)
    caminho_car.append(caminho)
vetores = caminho_edafo + caminho_car
for municipio in municipios:
    for vetor in vetores:
        if municipio in vetor and 'Aptd' in vetor:
            nome_arq = os.path.basename(vetor).replace('.shp', '_CAR.shp')
            uso = gpd.read_file(vetor)
        if municipio in vetor and 'AREA_IMOVEL' in vetor:
            area = gpd.read_file(vetor)
    base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
    base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{final_path}\{nome_arq}'))

C:\Users\YouX\AppData\Local\Temp\ipykernel_9184\2071217183.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4674

  base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
C:\Users\YouX\AppData\Local\Temp\ipykernel_9184\2071217183.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4674

  base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
C:\Users\YouX\AppData\Local\Temp\ipykernel_9184\2071217183.py:46: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:432